# Kapitel 3.5: Dekoratoren für das Klassendesign

* **`@property`:** Wir meistern den "Pythonic Way" für kontrollierten Attributzugriff und verstehen, wie Getter und Setter im Hintergrund funktionieren.
* **Eigene Dekoratoren in Klassen:** Wir wenden unsere selbstgebauten Dekoratoren auf Klassenmethoden an.
* **`@classmethod` und `@staticmethod`:** Wir analysieren den genauen Unterschied und die Anwendungsfälle für Methoden, die auf der Klasse bzw. losgelöst von der Instanz operieren.

---

## **1. `@property`: Kontrollierter und intelligenter Attributzugriff**

### **Das Problem mit dem direkten Zugriff**

Bisher haben wir auf Attribute direkt zugegriffen: `obj.attribut = wert`. Das ist einfach, aber unflexibel und potenziell unsicher.

* **Problem 1: Keine Validierung.** Was, wenn ein Gehalt nicht negativ oder eine Hüllen-Prozentzahl nicht über 100 sein darf? Bei direktem Zugriff gibt es keine Kontrolle.
* **Problem 2: Keine Flexibilität.** Was, wenn ein Attribut nicht direkt gespeichert, sondern aus anderen Attributen berechnet werden soll (z.B. `person.alter` berechnet aus `person.geburtsdatum`)?

Die traditionelle Lösung in anderen Sprachen sind `get()`- und `set()`-Methoden (z.B. `person.get_alter()`). Das ist in Python aber umständlich und gilt als "unpythonisch".



---
Szenario: Wir wollen eine Klasse Mitarbeiter erstellen. Das Attribut gehalt darf niemals auf einen negativen Wert gesetzt werden können.

Der unpythonische Weg (mit get_... und set_... Methoden)
Bei diesem Ansatz verstecken wir das eigentliche Attribut (z.B. als _gehalt) und zwingen den Nutzer der Klasse, spezielle Methoden für den Lese- und Schreibzugriff zu verwenden.

In [ ]:
class MitarbeiterUnpythonic:
    def __init__(self, name, startgehalt):
        self._gehalt = 0  # Initialwert
        self.name = name
        self.set_gehalt(startgehalt) # Validierung direkt bei Erstellung nutzen

    def get_gehalt(self):
        """Gibt den aktuellen Gehaltswert zurück."""
        return self._gehalt

    def set_gehalt(self, neuer_wert):
        self._gehalt = neuer_wert
        return self.get_gehalt()
        
# --- Anwendung ---
print("--- Unpythonischer Weg ---")
m1 = MitarbeiterUnpythonic("Anna", 40000)

# Zugriff und Änderung sind umständlich:
m1.set_gehalt(45000)
m1.set_gehalt(-5000)
print(f"Annas aktuelles Gehalt: {m1.get_gehalt()}")

**Die Pythonic-Lösung:** Der `@property`-Dekorator. Er erlaubt uns, den sauberen Zugriff über die Dot-Notation beizubehalten, im Hintergrund aber Logik auszuführen.

In [ ]:
class MitarbeiterPythonic:
    def __init__(self, name, startgehalt):
        self.name = name
        # Die Zuweisung hier ruft bereits den Setter auf!
        self.gehalt = startgehalt

    @property
    def gehalt(self):
        """Der Getter gibt einfach den internen Wert zurück."""
        return self._gehalt

    # 2. Der Setter wird mit dem Namen der Property + .setter definiert
    @gehalt.setter
    def gehalt(self, neuer_wert):
        self._gehalt = neuer_wert
        return self._gehalt
        

# --- Anwendung ---
print("\n--- Pythonic Way ---")
m2 = MitarbeiterPythonic("Ben", 50000)

# Zugriff und Änderung sehen aus wie bei einem normalen Attribut,
# rufen aber im Hintergrund den Getter und Setter auf.
m2.gehalt = 52000    # Ruft den Setter auf
m2.gehalt = -10000   # Ruft den Setter auf
print(f"Bens aktuelles Gehalt: {m2.gehalt}") # Ruft den Getter auf


---

### **Der große Vorteil eines Getters – Mehr als nur Werte abrufen**

Welchen Vorteil hat ein `@property`-Getter, wenn er nur `return self._attribut` macht, gegenüber dem direkten Zugriff?

Der Vorteil liegt nicht im Jetzt, sondern in der **Zukunftssicherheit und Flexibilität**

* **Konsistente öffentliche Schnittstelle (API):** Sie können mit einem einfachen öffentlichen Attribut starten. Wenn Sie später merken, dass Sie beim Lesen des Attributs eine Logik hinzufügen müssen (z.B. Logging, Berechnung), können Sie es in eine Property umwandeln. **Für den Nutzer Ihrer Klasse ändert sich nichts.** Er greift immer noch über `obj.attribut` zu. Ohne `@property` müssten Sie den Zugriff auf `obj.get_attribut()` ändern und damit den Code aller Nutzer Ihrer Klasse "brechen".
* **Berechnete Eigenschaften (Computed Properties):** Ein Getter muss nicht zwingend eine gespeicherte Variable zurückgeben. Er kann einen Wert "on-the-fly" aus anderen Attributen berechnen.

**Isoliertes Beispiel für einen berechneten Getter:**

In [1]:
# So könnte die 'property'-Klasse grob in Python implementiert sein:
class SimpleProperty:
    def __init__(self, fget=None, fset=None):
        self.fget = fget # Die Getter-Funktion (z.B. unsere 'gehalt'-Methode)
        self.fset = fset # Die Setter-Funktion

    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        # Ruft die bei der Erstellung übergebene Getter-Funktion auf
        return self.fget(instance)

    def __set__(self, instance, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        # Ruft die bei der Erstellung übergebene Setter-Funktion auf
        self.fset(instance, value)
        
    def setter(self, fset):
        # Diese Methode gibt eine NEUE Property-Instanz zurück,
        # die nun auch die Setter-Funktion kennt.
        return SimpleProperty(self.fget, fset)

# Wenn Sie @property schreiben, passiert im Grunde:
# gehalt = SimpleProperty(gehalt)

# Wenn Sie @gehalt.setter schreiben, passiert:
# gehalt = gehalt.setter(gehalt)


Das `@gehalt.setter`-Konstrukt funktioniert, weil der Aufruf von `@property` ein `property`-Objekt erzeugt, das selbst eine Methode namens `.setter` hat. Diese Methode nimmt die Setter-Funktion entgegen und erzeugt ein neues, vollständiges `property`-Objekt.


---

### **Beispiel 1: Getter und Setter mit Validierung**


In [2]:
class Mitarbeiter:
    def __init__(self, name, gehalt):
        self.name = name
        self._gehalt = gehalt # Internes Attribut
    
    # 1. Der Getter wird durch @property definiert
    @property
    def gehalt(self):
        print("-> GETTER: Greife auf Gehalt zu...")
        return self._gehalt

    # 2. Der Setter wird mit dem Namen der Property + .setter definiert
    @gehalt.setter
    def gehalt(self, neuer_wert):
        print(f"-> SETTER: Versuche, Gehalt auf {neuer_wert} zu setzen...")
        if not isinstance(neuer_wert, (int, float)) or neuer_wert < 30000:
            print("FEHLER: Ungültiger Gehaltswert! Muss eine Zahl >= 30000 sein.")
        else:
            self._gehalt = neuer_wert

m = Mitarbeiter("Ben", 50000)

# Ruft den Getter auf
print(f"Bens Gehalt: {m.gehalt}")

# Ruft den Setter auf (mit gültigem Wert)
m.gehalt = 55000 
print(f"Neues Gehalt: {m.gehalt}")

print("-" * 20)
# Ruft den Setter auf (mit ungültigem Wert)
m.gehalt = 25000
print(f"Gehalt nach ungültigem Versuch: {m.gehalt}")

-> GETTER: Greife auf Gehalt zu...
Bens Gehalt: 50000
-> SETTER: Versuche, Gehalt auf 55000 zu setzen...
-> GETTER: Greife auf Gehalt zu...
Neues Gehalt: 55000
--------------------
-> SETTER: Versuche, Gehalt auf 25000 zu setzen...
FEHLER: Ungültiger Gehaltswert! Muss eine Zahl >= 30000 sein.
-> GETTER: Greife auf Gehalt zu...
Gehalt nach ungültigem Versuch: 55000



---
### **Beispiel 2: Eine Kreis-Klasse mit gekoppelten Eigenschaften**

Hier nutzen wir Properties, um Attribute zu synchronisieren. Ändert man den Durchmesser, muss sich der Radius anpassen und umgekehrt.


In [3]:
import math

class Kreis:
    def __init__(self, radius):
        self.radius = radius

    @property
    def durchmesser(self):
        return self.radius * 2

    @durchmesser.setter
    def durchmesser(self, neuer_durchmesser):
        self.radius = neuer_durchmesser / 2
        
    @property
    def flaeche(self):
        # Eine berechnete, schreibgeschützte Eigenschaft
        return math.pi * self.radius ** 2

k = Kreis(10)
print(f"Radius: {k.radius}, Durchmesser: {k.durchmesser}, Fläche: {k.flaeche:.2f}")

# Ändern wir den Durchmesser, ändert sich der Radius automatisch mit
print("\nÄndere Durchmesser auf 50...")
k.durchmesser = 50
print(f"Neuer Radius: {k.radius}, Neuer Durchmesser: {k.durchmesser}, Neue Fläche: {k.flaeche:.2f}")

Radius: 10, Durchmesser: 20, Fläche: 314.16

Ändere Durchmesser auf 50...
Neuer Radius: 25.0, Neuer Durchmesser: 50.0, Neue Fläche: 1963.50


---
## Übung
---

### **Transfer auf das Projekt: Kontrollierte `Raumschiff`-Attribute**

Wir nutzen `@property`, um die `huelle_staerke` unseres Raumschiffs zu einem verwalteten Attribut zu machen. Es soll nicht möglich sein, die Hülle über einen Maximalwert zu "reparieren".


In [4]:
class Raumschiff:
    MAX_HUELLE = 100
    
    def __init__(self, name, huelle_staerke):
        self.name = name
        # Wichtig: Wir rufen hier den Setter auf, um die Validierung direkt zu nutzen!
        self.huelle_staerke = huelle_staerke

    @property
    def huelle_staerke(self):
        return self._huelle_staerke

    @huelle_staerke.setter
    def huelle_staerke(self, wert):
        if wert > self.MAX_HUELLE:
            self._huelle_staerke = self.MAX_HUELLE
            print(f"Hülle kann nicht über {self.MAX_HUELLE}% sein. Setze auf Maximum.")
        elif wert < 0:
            self._huelle_staerke = 0
        else:
            self._huelle_staerke = wert
            
enterprise = Raumschiff("Enterprise-D", 50)
print(f"Aktuelle Hülle: {enterprise.huelle_staerke}%")

print("\nRepariere um 80 Punkte...")
# Diese Zeile ruft den Getter auf (um 50 zu lesen), addiert 80, 
# und ruft dann den Setter mit dem Ergebnis 130 auf.
enterprise.huelle_staerke += 80 
print(f"Neue Hülle: {enterprise.huelle_staerke}%")

Aktuelle Hülle: 50%

Repariere um 80 Punkte...
Hülle kann nicht über 100% sein. Setze auf Maximum.
Neue Hülle: 100%



---
---

## **2. Eigene Dekoratoren in Klassen anwenden**

### **Funktionieren unsere selbstgebauten Dekoratoren auch bei Methoden?**

Eine Instanzmethode ist im Grunde eine Funktion, deren erster Parameter (`self`) automatisch von Python übergeben wird. Unser universeller Wrapper mit `*args` und `**kwargs` fängt diesen `self`-Parameter problemlos als erstes Element im `args`-Tupel auf.

### **Beispiel: Ein `@timer`-Dekorator für eine Klassenmethode**


In [5]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_zeit = time.time()
        ergebnis = func(*args, **kwargs)
        end_zeit = time.time()
        print(f"[{func.__name__}] Dauer: {end_zeit - start_zeit:.5f}s")
        return ergebnis
    return wrapper

class DatenbankHandler:
    def __init__(self, db_name):
        self.db_name = db_name
        
    @timer
    def daten_abfragen(self, query):
        print(f"Führe Query '{query}' auf Datenbank '{self.db_name}' aus...")
        time.sleep(1.5) # Simuliert eine langsame Datenbankabfrage
        return ["Ergebnis1", "Ergebnis2"]

db = DatenbankHandler("ProduktionDB")
resultate = db.daten_abfragen("SELECT * FROM users")
print(f"Abfrage lieferte: {resultate}")

Führe Query 'SELECT * FROM users' auf Datenbank 'ProduktionDB' aus...
[daten_abfragen] Dauer: 1.50332s
Abfrage lieferte: ['Ergebnis1', 'Ergebnis2']



---
---

## **3. `@classmethod` und `@staticmethod`: Werkzeuge der Klasse**

### **`@classmethod` – Methoden, die auf der Klasse arbeiten**

* **Was es ist:** Eine Methode, die als ersten Parameter automatisch die **Klasse selbst** (`cls`) erhält, nicht die Instanz (`self`).
* **Grund:**
    1.  **Alternative Konstruktoren:** Dies ist der häufigste und wichtigste Anwendungsfall. Manchmal ist es unpraktisch, ein Objekt immer nur über `__init__` zu erstellen. Vielleicht wollen Sie ein Objekt aus einem Dictionary, einem JSON-String oder einem Timestamp erzeugen. Ein `@classmethod` ist dafür die perfekte "Fabrik-Methode". Sie hält die Logik zur Objekterstellung sauber innerhalb der Klasse, anstatt sie auf externe Funktionen auszulagern.
    2.  **Arbeiten mit Klassenattributen:** Wenn eine Methode primär mit Klassenattributen arbeitet (z.B. einem globalen Zähler für alle Instanzen), ist ein `@classmethod` oft sauberer, da es klar signalisiert, dass kein Instanzzustand (`self`) benötigt wird.
* **Wie es im Hintergrund funktioniert (vereinfacht):** Man kann sich `@classmethod` als einen Deskriptor vorstellen, dessen `__get__`-Methode die Funktion nicht an die Instanz, sondern an die Klasse bindet, bevor sie zurückgegeben wird.

### **Beispiel: Eine `Datum`-Klasse mit alternativen Konstruktoren**

In [6]:
import datetime

class Datum:
    def __init__(self, tag, monat, jahr):
        self.tag = tag
        self.monat = monat
        self.jahr = jahr
        
    def __str__(self):
        return f"{self.tag:02d}.{self.monat:02d}.{self.jahr}"

    # Ein alternativer Konstruktor, der die Klasse 'cls' als Parameter erhält
    @classmethod
    def heute(cls):
        # 'cls' ist hier die Klasse 'Datum'
        heute = datetime.date.today()
        # Ruft den normalen __init__ der Klasse 'cls' auf
        return cls(heute.day, heute.month, heute.year)

    @classmethod
    def aus_iso_format(cls, iso_string): # z.B. "2025-06-30"
        jahr, monat, tag = map(int, iso_string.split('-'))
        return cls(tag, monat, jahr)

# Normaler Konstruktor
geburtstag = Datum(5, 10, 1990)
print(f"Geburtstag: {geburtstag}")

# Alternativer Konstruktor via @classmethod
heutiges_datum = Datum.heute()
print(f"Heute ist der: {heutiges_datum}")

# Zweiter alternativer Konstruktor via @classmethod
weihnachten = Datum.aus_iso_format("2025-12-24")
print(f"Weihnachten ist am: {weihnachten}")

Geburtstag: 05.10.1990
Heute ist der: 30.06.2025
Weihnachten ist am: 24.12.2025



---
### **`@staticmethod` – Normale Funktionen im Namensraum der Klasse**

* **Was es ist:** Im Grunde eine normale Funktion, die aber im "Namensraum" der Klasse "geparkt" ist, weil sie thematisch dorthin gehört.
* **Parameter:** Sie erhält **keinen** automatischen ersten Parameter (weder `self` noch `cls`).
* **Wieso brauche ich das?:** Hauptsächlich zur **Code-Organisation und Lesbarkeit**. Wenn Sie eine Hilfsfunktion haben, die logisch zu einer Klasse gehört, aber keinen Zugriff auf Klassen- oder Instanzvariablen benötigt (z.B. eine Validierungs- oder Konvertierungsfunktion), ist es sauberer, sie als `@staticmethod` in die Klasse zu packen, anstatt sie als globale Funktion im Modul "herumfliegen" zu lassen.
* **Wie es im Hintergrund funktioniert:** Dies ist der einfachste der drei Dekoratoren. Er sorgt im Grunde nur dafür, dass Python beim Aufruf nicht versucht, `self` als ersten Parameter zu übergeben.

### **Beispiel: `Datum`-Klasse mit Utility-Funktion**

class Datum:
    # ... (init und classmethods von oben) ...

    # Eine Utility-Funktion, die keinen Zustand braucht
    @staticmethod
    def ist_schaltjahr(jahr):
        # Diese Logik hängt nicht von einer spezifischen Instanz oder Klasse ab
        return jahr % 4 == 0 and (jahr % 100 != 0 or jahr % 400 == 0)

# Man ruft sie direkt über die Klasse auf
print(f"War 2024 ein Schaltjahr? {Datum.ist_schaltjahr(2024)}")
print(f"War 1990 ein Schaltjahr? {Datum.ist_schaltjahr(1990)}")


---

## Übung



---

### **Transfer auf das Projekt: `Raumschiff`-Fabriken und -Validatoren**


In [ ]:
class Raumschiff:
    def __init__(self, name, klasse, huelle):
        # Die Validierung wird jetzt direkt hier genutzt
        if not self.validiere_kennung(name):
             raise ValueError(f"Ungültige Schiffskennung: '{name}'")
        self.name = name
        self.klasse = klasse
        self.huelle = huelle
    
    def __str__(self):
        return f"{self.klasse}-Klasse '{self.name}' (Hülle: {self.huelle}%)"
    
    # Ein alternativer Konstruktor
    @classmethod
    def baue_standard_fregatte(cls, name):
        # Ruft den normalen __init__ der Klasse 'cls' mit vordefinierten Werten auf
        return cls(name, "Fregatte", 100)

    # Eine thematisch passende Utility-Funktion
    @staticmethod
    def validiere_kennung(kennung):
        return kennung.startswith("USS-") and len(kennung) > 4

# Erstellen via @classmethod
fregatte = Raumschiff.baue_standard_fregatte("USS-Reliant")
print(f"Neue Fregatte gebaut: {fregatte}")

# Direkter Aufruf der @staticmethod zur Validierung
is_valid = Raumschiff.validiere_kennung("NCC-1701")
print(f"Ist 'NCC-1701' eine gültige Kennung? {is_valid}")

Neue Fregatte gebaut: Fregatte-Klasse 'USS-Reliant' (Hülle: 100%)
Ist 'NCC-1701' eine gültige Kennung? False



---
---

## **5. Tages-Challenge: Anwendung im Schüler-Projekt (`Smart Grid`)**

Jetzt kombinieren wir alle heutigen Konzepte in Ihrem Projekt.

**Ihre umfassende Aufgabe:**

1.  **Schreiben Sie einen Dekorator:**
    * Erstellen Sie einen Dekorator `@log_status_aenderung`, der vor dem Aufruf einer Methode ausgibt `"[LOG] Status von '{self.name}' wird geändert..."` und danach `"[LOG] Statusänderung abgeschlossen."`. Er muss mit Methoden von Objekten funktionieren.
2.  **`@property` implementieren:**
    * Das Attribut `aktuelle_leistung` in Ihrem `Energieerzeuger` soll zu einer verwalteten Eigenschaft werden.
    * Der **Getter** soll einfach den Wert zurückgeben.
    * Der **Setter** soll sicherstellen, dass die `aktuelle_leistung` niemals größer als die `max_leistung` oder kleiner als 0 sein kann. Bei ungültigen Werten soll eine Warnung ausgegeben und der Wert auf den nächstgelegenen gültigen Wert (0 oder `max_leistung`) gesetzt werden.
3.  **Dekorator anwenden:**
    * Wenden Sie Ihren neuen `@log_status_aenderung`-Dekorator auf den **Setter** der `aktuelle_leistung`-Property an. (Ja, das ist möglich!)
4.  **`@classmethod` und `@staticmethod` hinzufügen:**
    * Fügen Sie Ihrer `Solaranlage`-Klasse eine `@classmethod` namens `erzeuge_farm_aus_gesamtfleache(cls, name, max_leistung_pro_qm, gesamtflaeche)` hinzu. Diese Methode soll die `max_leistung` aus der Fläche berechnen und eine neue Instanz der `Solaranlage` zurückgeben.
    * Fügen Sie Ihrer Basisklasse `Energieerzeuger` eine `@staticmethod` namens `ist_netz_stabil(frequenz)` hinzu. Sie soll `True` zurückgeben, wenn die übergebene `frequenz` zwischen 49.8 und 50.2 Hz liegt, andernfalls `False`.
5.  **Alles testen:**
    * Erstellen Sie eine Instanz und rufen Sie den Setter für `aktuelle_leistung` auf, um die Log-Ausgabe des Dekorators zu sehen.
    * Erstellen Sie eine neue Solarfarm mit Ihrer `@classmethod`.
    * Testen Sie die Netzstabilität mit Ihrer `@staticmethod`.
